In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
import os


In [4]:
dataset = load_dataset(
    "json",
    data_files={
        "train": "train.jsonl",
        "validation": "test.jsonl"
    } 
)

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 500
    })
    validation: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 100
    })
})

In [21]:
checkpoint = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

mapDict = {
    "suporte": 0,
    "venda": 1
}

def transform_labels(example):
    completion_value = example["completion"]
    return {"labels": mapDict[completion_value]}

def tokenize_function(example):
    return tokenizer(example["prompt"], truncation=True, padding=True)

In [22]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.map(transform_labels)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [23]:
from transformers import TrainingArguments

output_dir = "./bert-validator-test"
os.makedirs(output_dir, exist_ok=True)
os.makedirs("./logs", exist_ok=True)

training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,

    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4,

    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_steps=100,


    eval_strategy="steps",
    eval_steps=200,

    save_strategy="steps",
    save_steps=200,
    save_total_limit=3,

    load_best_model_at_end=True,
    metric_for_best_model="accuracy",   # ou "f1", "eval_loss"
    greater_is_better=True,

    logging_dir="./logs",
    logging_steps=100,
    logging_strategy="steps",

    report_to="none",
    fp16=True,
    dataloader_num_workers=2,
    seed=42,

    disable_tqdm=False,
)

In [24]:
from transformers import Trainer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "offline"

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [26]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

C:\Users\aleos\AppData\Local\Temp\ipykernel_3860\2871184925.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [27]:
trainer.train()

c:\Users\aleos\Documents\Projetos\Python\project\fine_tunning\models\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss,Validation Loss


TrainOutput(global_step=48, training_loss=0.6403200229008993, metrics={'train_runtime': 314.8772, 'train_samples_per_second': 4.764, 'train_steps_per_second': 0.152, 'total_flos': 20041842366000.0, 'train_loss': 0.6403200229008993, 'epoch': 3.0})

In [28]:
trainer.evaluate()

c:\Users\aleos\Documents\Projetos\Python\project\fine_tunning\models\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.07392331957817078,
 'eval_accuracy': 1.0,
 'eval_runtime': 61.6515,
 'eval_samples_per_second': 1.622,
 'eval_steps_per_second': 0.211,
 'epoch': 3.0}

In [29]:
trainer.save_model()

In [32]:
from transformers import pipeline

# Carregar o modelo treinado
classifier = pipeline("text-classification", model=output_dir, tokenizer=tokenizer)

# Mapeamento inverso para exibir os nomes das classes
label_names = {0: "suporte", 1: "venda"}


Device set to use cpu


In [33]:
# Função para testar um único texto
def classificar_texto(texto):
    resultado = classifier(texto)[0]
    label_id = int(resultado['label'].split('_')[-1])
    classe_predita = label_names.get(label_id, resultado['label'])
    confianca = resultado['score']
    
    return {
        'texto': texto,
        'classe': classe_predita,
        'confianca': confianca
    }

# Exemplo de uso
texto_novo = "Quero comprar o produto mas tenho dúvidas sobre o preço"
resultado = classificar_texto(texto_novo)
print(f"Texto: {resultado['texto']}")
print(f"Classificação: {resultado['classe']}")
print(f"Confiança: {resultado['confianca']:.2%}")


Texto: Quero comprar o produto mas tenho dúvidas sobre o preço
Classificação: suporte
Confiança: 91.69%


In [35]:
# Textos de exemplo para teste
textos_teste = [
    "Boa tarde! Estou querendo comprar o fone JBL Flip 6 preto pra dar de presente de Natal. Vocês estão com alguma promoção ou cupom de desconto ativo?",
    "Comprei uma cafeteira da marca X há 15 dias e ela começou a vazar água pela lateral. Já limpei tudo e continua acontecendo. Como faço pra acionar a garantia?",
    "Boa noite. O ar-condicionado split 12000 BTUs que instalei semana passada está fazendo um barulho estranho quando desliga, parece um estalo bem forte. Isso é normal ou preciso chamar assistência?",
    "Oi, tudo bem? Vi no site de vocês o notebook Gamer com RTX 4070. Ainda tem em estoque? Qual o prazo de entrega pra São Paulo capital e dá pra parcelar em 12x sem juros?",
    "Tenho um problema técnico com o sistema"
]

# Fazer predições
for texto in textos_teste:
    resultado = classifier(texto)[0]
    label_id = int(resultado['label'].split('_')[-1])
    classe_predita = label_names.get(label_id, resultado['label'])
    confianca = resultado['score']
    
    print(f"Texto: {texto}")
    print(f"Classificação: {classe_predita}")
    print(f"Confiança: {confianca:.4f}")
    print("-" * 80)


Texto: Boa tarde! Estou querendo comprar o fone JBL Flip 6 preto pra dar de presente de Natal. Vocês estão com alguma promoção ou cupom de desconto ativo?
Classificação: venda
Confiança: 0.9155
--------------------------------------------------------------------------------
Texto: Comprei uma cafeteira da marca X há 15 dias e ela começou a vazar água pela lateral. Já limpei tudo e continua acontecendo. Como faço pra acionar a garantia?
Classificação: suporte
Confiança: 0.9158
--------------------------------------------------------------------------------
Texto: Boa noite. O ar-condicionado split 12000 BTUs que instalei semana passada está fazendo um barulho estranho quando desliga, parece um estalo bem forte. Isso é normal ou preciso chamar assistência?
Classificação: suporte
Confiança: 0.8334
--------------------------------------------------------------------------------
Texto: Oi, tudo bem? Vi no site de vocês o notebook Gamer com RTX 4070. Ainda tem em estoque? Qual o prazo de ent